### pySpark ml word2vec

本篇是使用 pyspark 進行 word2vec 進行訓練。可以學到

* pyspark word2vec 的建模方式。
* 將 vector 轉為 csv 的儲存方式。

In [1]:
# 設定spark session 環境變數 spark home
# 詳細 ?findspark.init
import findspark
findspark.init('/usr/local/spark')
# 載入必要modules/
from pyspark.sql import SparkSession
from pyspark.sql import functions as fn
from pyspark.ml.feature import Word2Vec, Word2VecModel

In [2]:
hdfs_path = "hdfs://HDFS_IP:PORT"

# 開啟 spark session
# pyspark session 目前需要手動設定使用到的參數
# spark.executor.memory worker 計算/cache相關記憶體大小
# spark.driver.memory 與 driver 相關的記憶體大小影響task分派，對演算速度影響大
# spark.driver.maxResultSize 設定分散資料collect回本地端的大小上限，SparkR有更改設定檔無限制，此處須設置。
ss = SparkSession.builder \
    .master( "spark://MASTER_IP:PORT" ) \
    .appName( "word_to_vector" ) \
    .config( "spark.cores.max", "8" ) \
    .config( "spark.executor.memory", "32g" ) \
    .config( "spark.driver.memory", "24g" ) \
    .config( "spark.driver.maxResultSize", "16g" ) \
    .config( "spark.rpc.message.maxSize", "512") \
    .getOrCreate() 

In [3]:
# 讀取 parquet 資料源
corpus = ss.read.parquet( hdfs_path + "/corpus/external/cut/*" )

In [4]:
# 設定 word2vec 參數，並進行訓練和儲存模型
# numPartitions 使用和 core 數相同，maxIter 數官方文件建議須小於等於 partition 數
# maxSentenceLength 設定為 8000，我們 corpus 的長度都超過 1000，預設數值明顯不符合需求。需要確認文章長度平均值及測試。
word2Vec = Word2Vec( vectorSize = 240, 
                     inputCol = "words", 
                     outputCol = "word2vec",
                     windowSize = 6,
                     numPartitions = 8,
                     stepSize = 0.05, 
                     maxIter = 8,
                     maxSentenceLength = 8000 )
w2v_model = word2Vec.fit( corpus )
w2v_model.write().overwrite().save( hdfs_path + "/corpus/ml/WordToVector_model" )

In [3]:
# 讀取模型
w2v_model = Word2VecModel.load( hdfs_path + "/corpus/ml/WordToVector_model" )

In [4]:
# 取出 word embedding 另外儲存，供後續 sparkR 分析使用
w2v_vectors = w2v_model.getVectors()
w2v_vectors = w2v_vectors.rdd.map( lambda row: ( row["word"], ) + tuple( row["vector"].toArray().tolist() ) )\
                         .toDF( ["word"] )
w2v_vectors.write.parquet( path = hdfs_path + "/corpus/ml/WordToVector_embedding", 
                           mode = "overwrite",
                           compression = "gzip" )

In [45]:
# 嘗試 word2vec 訓練成效
w2v_model.findSynonyms("switch", 10).show()

+-----------+------------------+
|       word|        similarity|
+-----------+------------------+
|   nintendo|0.7870251536369324|
|        wii|0.7406870722770691|
|        3ds|0.7273473143577576|
|        任天堂|0.7169155478477478|
|       xbox| 0.696182906627655|
|        nds|0.6801103949546814|
|playstation|0.6680821776390076|
|   gamecube| 0.667585551738739|
|    console|0.6661582589149475|
|         主機|0.6646143794059753|
+-----------+------------------+



In [46]:
ss.stop()